In [0]:
#Montando ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [0]:
uf = input("Entre com a Unidade Federativa desejada: ")

In [0]:
#Lendo base de dados do Google Drive
dataSet = pd.read_csv("/content/drive/My Drive/CCD/DataToLSTM/" + uf + ".csv").drop(columns = "Unnamed: 0")

In [0]:
#Cidades identificadas na UF escolhida
cidades = dataSet["name"].drop_duplicates()

In [0]:
print(cidades)

In [0]:
#Escolha aleatória da cidade no Estado escolhido
cidadeAleatoria = cidades.sample()
print(cidadeAleatoria)

In [0]:
#Colunas da base de dados
dataSet.columns

In [0]:
#Cabeçalho dos dados
dataSet.head(30)

In [0]:
#Convertendo datas para objeto "DateTime"
dataSet["date"] = pd.to_datetime(dataSet["date"], format="%Y-%m-%d %H:%M:%S")

In [0]:
#Selecionando dados para a cidade escolhida, eliminando variáveis "id", "name" e "uf", que não serão mais importantes
dataTreino = dataSet[dataSet["name"] == cidadeAleatoria.values[0]].drop(columns = ["id","name","uf"]).reset_index(drop = True)

In [0]:
#Colunas para base de dados para treino
dataTreino.columns

In [0]:
#Cabeçalho dos dados
dataTreino.head(30)

In [0]:
#Criando uma lista "keyAux" para guardar todos os dias (sem horários) existentes na base de dados
keyAux = []
for i in dataTreino["date"].values:
  aux = pd.to_datetime(i).date()
  keyAux.append(aux)

In [0]:
#Transformando os dados de uma base de dados horária para dados diários, tornando possível a modelagem do problema
dataTreino["dataIndex"] = pd.Series(keyAux)
dataTrain = pd.DataFrame(index = pd.Series(keyAux).drop_duplicates())
dados = ['prec', 'tair', 'tw', 'tmax', 'tmin', 'urmax', 'patm', 'pnmm', 'wd', 'wsmax', 'n', 'cc', 'evap', 'ur', 'ws']
for i in dados:
  dataTrain[i] = np.nan
for i in dataTrain.index:
  aux = dataTreino[dataTreino["dataIndex"] == i][dados]
  for j in dados:
    dataTrain.loc[i, j] = max(aux[j])

In [0]:
#Eliminando dados em que não exista uma medida de velocidade do vento existente -> sendo, portanto, impossível de comparar o resultado
dataTrain = dataTrain.dropna(subset = ["ws"]).reset_index(drop = True)

In [0]:
#Descrição estatística para base de dados
dataTrain.describe()

In [0]:
#Separando velocidade do vento da base de dados
velVento = dataTreino["ws"]
dataTrain = dataTrain.drop(columns = ["ws"])

In [0]:
#Cabeçalho de nova base de dados
dataTrain.head()

In [0]:
#Fazendo uma transformação por Score-Z
sc = StandardScaler()
dataTrain = pd.DataFrame(sc.fit_transform(dataTrain), columns = dataTrain.columns)

In [0]:
#Nova descrição nos dados
dataTrain.describe()

In [0]:
#Transformando "NaN" em 0 (nesse caso, o 0 se torna não prejudicial para o modelo de rede neural utilizado, devido a função de ativação ser do tipo tanh)
dataTrain = dataTrain.fillna(0)

In [0]:
#Verificando o grau de informação mantido, ao reduzir o problema de 14 dimensões para 10 dimensões através de PCA
pca = PCA(14)
pca.fit(dataTrain)
autovalores = pca.explained_variance_
print("Grau de informação preservado: " + str(sum(autovalores[:10]) / sum(autovalores)))

In [0]:
#Reduzindo para 10 dimensões a base de dados através de PCA
pca = PCA(10)
dadosTransformados = pd.DataFrame(pca.fit_transform(dataTrain), columns = ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"])

In [0]:
#Dimensão de nova base de dados
dadosTransformados.shape

In [0]:
#Colocando a velocidade do vento de volta na base de dados
dadosTransformados = dadosTransformados.join(velVento)

In [0]:
#Cabeçalho de nova base de dados
dadosTransformados.head()

In [0]:
#Descrição de nova base de dados
dadosTransformados.describe()

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [0]:
#Preparando dados para a LSTM
x_train, y_train = [], []
for i in range(3,len(dadosTransformados)):
    local = dadosTransformados.iloc[(i-3):(i-1)]
    local = local.append(dadosTransformados.loc[i - 1, ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10"]])
    x_train.append(np.array(local))
    ws = dadosTransformados.loc[i - 1, "ws"]
    y_train.append(np.array(ws))

In [0]:
#Transformando x_train e y_train em numpyArray
x_train, y_train = np.array(x_train), np.array(y_train)

In [0]:
#Dimensões da base de dados de entrada para treino
x_train.shape

In [0]:
#Dimensões da base de dados de saída para treino
y_train.shape

In [0]:
#Substituindo NaN por 0
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train).reshape(-1,1)

In [0]:
#Separando 10% de dados para teste e 90% para treino
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.1, random_state = 15)

In [0]:
#Criando o modelo de previsão do tipo sequential, com uma rede neural do tipo LSTM
regressor = Sequential()
regressor.add(LSTM(activation = "tanh", units = 30, return_sequences = True, input_shape = (x_train.shape[1], x_train.shape[2])))
regressor.add(LSTM(activation = "tanh", units = 30, return_sequences = True))
regressor.add(LSTM(activation = "tanh", units = 30))
regressor.add(Dense(units = y_train.shape[1]))

In [0]:
#Compilando a rede neural
regressor.compile(optimizer = "adam", loss='mse', metrics=['mae'])

In [0]:
#Treinamento com 30 epochs
hist = regressor.fit(x_train, y_train, epochs = 30, validation_data=(x_test, y_test))

In [0]:
#Avaliando o MAE, RMSE e o Coef. de Corr. de Pearson (R)
print("MAE : " + str(mean_absolute_error(y_test[:,0], regressor.predict(x_test)[:,0])))
print("RMSE : " + str((mean_squared_error(y_test[:,0], regressor.predict(x_test)[:,0])) ** 0.5))
print("R : " + str(np.corrcoef(y_test[:,0], regressor.predict(x_test)[:,0])[0][1]))